In [121]:
from random import randint
from hashlib import sha3_256

class DOT():
    def __init__(self, x, y, p=None):
        self.x = x
        self.y = y
        self.p = p

    @property
    def info(self):
        print(self.x, self.y)

    def __str__(self):
        return "(" + str(self.x) + ", " + str(self.y) + ")"

class EC():
    def __init__(self, a, b, p) -> None:
        #y^2 = x^3 + a*x +b mod p
        self.a = a
        self.b = b
        self.p = p

    def dop_info(self, n,h) -> None:
        self.n = n
        self.h = h

    def add_seed(self, dot):
        self.seed = dot

    def modInverse(self, x):
        return self.p - x

    def add(self, a, b):
        g = DOT(a.x, a.y)
        dx = (a.x - b.x) % self.p
        dy = (a.y - b.y) % self.p
        m = dy * (-dx % self.p) % self.p
        g.x = (m*m -a.x -b.x)%self.p
        g.y = (m * (a.x - g.x) - a.y) % self.p
        return g

    def __mul__(self,  N):
        #if a is None:
        a = DOT(self.seed.x, self.seed.y)
        b = DOT(a.x,a.y)
        for i in range(N):
            b = self.add(a,b)
        return b    

    def mul(self, a, N):
        #if a is None:
        #    a = DOT(self.seed.x, self.seed.y)
        b = DOT(a.x,a.y)
        for i in range(N):
            b = a + b
        return b 

    @property
    def info(self):
        print("y^2 = x^3 + x *(", a,") + (", b,") mod ", p)
        print("Порядок подгруппы: ", n)
        print("Кофактор подгруппы: ", h)
        print("seed: ", seed)

    def find_seed(self, seed = 0):
        if seed == 0:
            pass

class Crypt():
    def __init__(self, ec, start):
        self.ec = ec
        self.ec.seed=start
    
    def key_gen(self, priv):
        self.priv = priv
        hash_object = sha3_256(priv)
        hex_dig = hash_object.hexdigest()
        self.hash_priv = hex_dig, 16
        N = int(hex_dig, 16)%10**6
        self.pub = self.ec*N
        return self.pub
    
    @property
    def info(self):
        self.ec.info
        print("priv: ", self.priv)
        print("priv hash: ", self.hash_priv)
        print("pub: ", self.pub)

# В качестве параметров кривой взята первая рекомендация из NIST

In [122]:
a = -3
b = 2455155546008943817740293915197451784769108058161191238065
p = 6277101735386680763835789423207666416083908700390324961279
x = 602046282375688656758213480587526111916698976636884684818
y = 174050332293622031404857552280219410364023488927386650641 
n = 6277101735386680763835789423176059013767194773182842284081 
h = 1

curv = EC(a,b,p) 
curv.dop_info(n, h)
seed = DOT(x,y) 
curv.add_seed(seed)

k = randint(0,30)
start = curv * k
channel = Crypt(curv, start)
priv = b'Something'
pub = channel.key_gen(priv)
channel.info


y^2 = x^3 + x *( -3 ) + ( 2455155546008943817740293915197451784769108058161191238065 ) mod  6277101735386680763835789423207666416083908700390324961279
Порядок подгруппы:  6277101735386680763835789423176059013767194773182842284081
Кофактор подгруппы:  1
seed:  (602046282375688656758213480587526111916698976636884684818, 174050332293622031404857552280219410364023488927386650641)
priv:  b'Something'
priv hash:  ('64e335d39d86e261cd0d4af21b0dc065e6f95003c110046a2f7d06a0c86c5008', 16)
pub:  (2463963105483528461606179672039874467760713695432813650182, 1700807908897289281540939918015282584862971282808272144954)
